# Error Analysis

In [45]:
import pandas as pd
import spacy
import thesis_utils
from IPython.display import display, HTML
from collections import defaultdict
from tqdm import tqdm
import time
from openai import OpenAI, APITimeoutError, APIConnectionError, APIError
from openai import APITimeoutError
import random
import os
import csv

In [47]:
xlm_roberta_fce = './predictions_on_processedfiles/proc_fce_xlm_roberta.tsv'

In [49]:
xlm_roberta_fce_sents = thesis_utils.read_tsv_file_and_find_sentences_with_headers(xlm_roberta_fce)

In [51]:
def evaluate_error_types_recall_only_merlin(sentences, dataset, print_results=True):
    error_stats = defaultdict(lambda: {"gold_i": 0, "correct_pred_i": 0})
    
    for sentence in sentences:
        for line in sentence:
            if dataset == 'merlin':
                error_type = line[6]
                gold_label = line[-2]
                pred_label = line[-1]
                if gold_label == 'i':
                    error_stats[error_type]["gold_i"] += 1
                    if pred_label == 'i':
                        error_stats[error_type]["correct_pred_i"] += 1
            elif dataset == 'fce':
                error_type = line[3]
                gold_label = line[-2]
                pred_label = line[-1]
                if gold_label == 'i':
                    error_stats[error_type]["gold_i"] += 1
                    if pred_label == 'i':
                        error_stats[error_type]["correct_pred_i"] += 1
            elif dataset == 'realec':
                gold_label = line[-2]
                pred_label = line[-1]
                error_type = line[2]
                if gold_label == 'i':
                    error_stats[error_type]["gold_i"] += 1
                    if pred_label == 'i':
                        error_stats[error_type]["correct_pred_i"] += 1

    if print_results:
        header = f"| {'Error Code':<23} | {'Gold':>6} | {'True Pos.':>7} | {'False Neg.':>10} | {'Recall':>6} |"
        print(header)
        print("|" + "-"*(len(header)-2) + "|")

        total_gold = 0
        total_correct = 0

        sorted_items = sorted(
            error_stats.items(), 
            key=lambda item: item[1]["correct_pred_i"] / item[1]["gold_i"] if item[1]["gold_i"] > 0 else 0.0, 
            reverse=True
        )

        for error_type, stats in sorted_items:
            gold_i = stats["gold_i"]
            correct_pred_i = stats["correct_pred_i"]
            false_neg = gold_i - correct_pred_i
            recall = correct_pred_i / gold_i if gold_i > 0 else 0.0
            print(f"| {error_type:<23} | {gold_i:6d} | {correct_pred_i:9d} | {false_neg:10d} | {recall:6.2f} |")
            
            total_gold += gold_i
            total_correct += correct_pred_i

        total_false_neg = total_gold - total_correct
        overall_recall = total_correct / total_gold if total_gold > 0 else 0.0

        print("|" + "-"*(len(header)-2) + "|")
        print(f"| {'TOTAL':<23} | {total_gold:6d} | {total_correct:9d} | {total_false_neg:10d} | {overall_recall:6.2f} |")

    return error_stats

In [53]:
roberta_fce_stats = evaluate_error_types_recall_only_merlin(xlm_roberta_fce_sents,'fce')

| Error Code              |   Gold | True Pos. | False Neg. | Recall |
|--------------------------------------------------------------------|
| CN                      |      9 |         9 |          0 |   1.00 |
| IV                      |     30 |        30 |          0 |   1.00 |
| IQ                      |      2 |         2 |          0 |   1.00 |
| IA                      |      2 |         2 |          0 |   1.00 |
| MV                      |      1 |         1 |          0 |   1.00 |
| DA                      |     15 |        14 |          1 |   0.93 |
| S                       |    331 |       304 |         27 |   0.92 |
| IN                      |     11 |        10 |          1 |   0.91 |
| AGD                     |      7 |         6 |          1 |   0.86 |
| DV                      |      7 |         6 |          1 |   0.86 |
| MD                      |      8 |         6 |          2 |   0.75 |
| DD                      |      4 |         3 |          1 |   0.75 |
| FY  

In [55]:
def find_false_positives_and_false_negatives(list_sentences, dataset):
    only_fp = []
    only_fn = []
    both_fp_fn = []

    false_negatives_by_type_clean = defaultdict(set)
    false_negatives_by_type_all = defaultdict(set)

    for sentence in list_sentences:
        has_fp = any(token[-1] == 'i' and token[-2] == 'c' for token in sentence)
        has_fn = any(token[-1] == 'c' and token[-2] == 'i' for token in sentence)

        if has_fp and not has_fn:
            only_fp.append(sentence)
        elif has_fn and not has_fp:
            only_fn.append(sentence)
        elif has_fp and has_fn:
            both_fp_fn.append(sentence)

    for sentence in only_fn:
        if dataset == 'merlin':
            for token in sentence:
                if token[-1] == 'c' and token[-2] == 'i' and len(token) > 6:
                    error_type = token[6] if token[6] else 'NO_LABEL'
                    false_negatives_by_type_clean[error_type].add(tuple(tuple(t) for t in sentence))
        elif dataset == 'fce':
            for token in sentence:
                if token[-1] == 'c' and token[-2] == 'i' and len(token) > 3:
                    error_type = token[3] if token[3] else 'NO_LABEL'
                    false_negatives_by_type_clean[error_type].add(tuple(tuple(t) for t in sentence))
        else:
            for token in sentence:
                if token[-1] == 'c' and token[-2] == 'i':
                    error_type = token[2] if token[2] else 'NO_LABEL'
                    false_negatives_by_type_clean[error_type].add(tuple(tuple(t) for t in sentence))

    for sentence in both_fp_fn:
        if dataset == 'merlin':
            for token in sentence:
                if token[-1] == 'c' and token[-2] == 'i' and len(token) > 6:
                    error_type = token[6] if token[6] else 'NO_LABEL'
                    false_negatives_by_type_all[error_type].add(tuple(tuple(t) for t in sentence))
        elif dataset == 'fce':
            for token in sentence:
                if token[-1] == 'c' and token[-2] == 'i' and len(token) > 3:
                    error_type = token[3] if token[3] else 'NO_LABEL'
                    false_negatives_by_type_all[error_type].add(tuple(tuple(t) for t in sentence))
        else:
            for token in sentence:
                if token[-1] == 'c' and token[-2] == 'i':
                    error_type = token[2] if token[2] else 'NO_LABEL'
                    false_negatives_by_type_all[error_type].add(tuple(tuple(t) for t in sentence))

    false_negatives_by_type_clean = {k: list(v) for k, v in false_negatives_by_type_clean.items()}
    false_negatives_by_type_all = {k: list(v) for k, v in false_negatives_by_type_all.items()}

    return only_fp, only_fn, both_fp_fn, false_negatives_by_type_clean, false_negatives_by_type_all

roberta_fce_only_fp, roberta_fce_only_fn, roberta_fce_fp_and_fn, roberta_fce_only_fn_error_type, roberta_fce_fn_fp_error_type = find_false_positives_and_false_negatives(xlm_roberta_fce_sents, 'fce')
print(f"False Positive Sentences (roberta-fce): {len(roberta_fce_only_fp)}")
print(f"False Negative Sentences (roberta-fce): {len(roberta_fce_only_fn)}\n")

False Positive Sentences (roberta-fce): 129
False Negative Sentences (roberta-fce): 703



In [13]:
for key in sorted(set(roberta_fce_only_fn_error_type) | set(roberta_fce_fn_fp_error_type)):
    fn_count = len(roberta_fce_only_fn_error_type.get(key, []))
    fp_fn_count = len(roberta_fce_fn_fp_error_type.get(key, []))
    print(f"| {key:<18} | FN: {fn_count:6d} | TOTAL: {fp_fn_count:6d} |")

| AGA                | FN:      2 | TOTAL:      2 |
| AGD                | FN:      1 | TOTAL:      0 |
| AGN                | FN:     19 | TOTAL:      8 |
| AGQ                | FN:      1 | TOTAL:      0 |
| AGV                | FN:     10 | TOTAL:      3 |
| AS                 | FN:     13 | TOTAL:      3 |
| CQ                 | FN:      2 | TOTAL:      3 |
| DA                 | FN:      0 | TOTAL:      1 |
| DD                 | FN:      1 | TOTAL:      0 |
| DJ                 | FN:     14 | TOTAL:      2 |
| DN                 | FN:      6 | TOTAL:      2 |
| DT                 | FN:      0 | TOTAL:      1 |
| DV                 | FN:      0 | TOTAL:      1 |
| DY                 | FN:      5 | TOTAL:      3 |
| FD                 | FN:      4 | TOTAL:      2 |
| FJ                 | FN:      1 | TOTAL:      0 |
| FN                 | FN:     21 | TOTAL:      8 |
| FV                 | FN:     32 | TOTAL:      7 |
| FY                 | FN:      1 | TOTAL:      0 |
| ID        

In [69]:
def visualize_aligned_tokens_aligned(data):
    for i, sentence in enumerate(data):
        print(f"\n=== Sentence {i+1} ===")

        tokens = [tok[1] for tok in sentence]
        corrections = [tok[2] for tok in sentence]
        error_types = [tok[3] if tok[3] else "-" for tok in sentence]
        golds = [tok[4] for tok in sentence]
        preds = [tok[-1] for tok in sentence]

        col_widths = [max(len(tok), len(corr), len(err), len(g), len(p))
                      for tok, corr, err, g, p in zip(tokens, corrections, error_types, golds, preds)]
        padded = lambda row: " | ".join(f"{item:<{w}}" for item, w in zip(row, col_widths))

        print("TOKENS:       ", padded(tokens))
        print("CORRECTIONS:  ", padded(corrections))
        print("ERROR TYPES:  ", padded(error_types))
        print("GOLD LABELS:  ", padded(golds))
        print("PRED LABELS:  ", padded(preds))




visualize_aligned_tokens_aligned(roberta_fce_only_fp)


=== Sentence 1 ===
TOKENS:        Dear | Principal | , | First | of | all | , | we | would | like | to | thank | you | for | good | programme | which | provides | us | 3 | days  | stay | in | London | .
CORRECTIONS:        |           |   |       |    |     |   |    |       |      |    |       |     |     |      |           |       |          |    |   | days' |      |    |        |  
ERROR TYPES:   -    | -         | - | -     | -  | -   | - | -  | -     | -    | -  | -     | -   | -   | -    | -         | -     | -        | -  | - | MP    | -    | -  | -      | -
GOLD LABELS:   c    | c         | c | c     | c  | c   | c | c  | c     | c    | c  | c     | c   | c   | i    | c         | c     | c        | c  | i | c     | c    | c  | c      | c
PRED LABELS:   c    | c         | c | c     | c  | c   | c | c  | c     | c    | c  | c     | c   | c   | i    | c         | c     | c        | c  | i | i     | i    | c  | c      | c

=== Sentence 2 ===
TOKENS:        However | , | we | have |

In [ ]:
def save_sentences_to_tsv(data, error_type, file_path):
    mode = 'a' if os.path.exists(file_path) else 'w'

    with open(file_path, mode, encoding='utf-8', newline='') as f:
        writer = csv.writer(f, delimiter='\t')

        if mode == 'w':
            writer.writerow(["error_type", "sentence"])

        for sentence in data:
            tokens = [tok[1] for tok in sentence]
            sentence_text = " ".join(tokens)
            writer.writerow([error_type, sentence_text])

In [75]:
save_sentences_to_tsv(roberta_fce_only_fp[:50], 'FP', 'classified_sentencesFP_fce.tsv')

# Qwen!!!

In [10]:
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

##  Prompt Mode

In [33]:
def query_LLM(model_client, prompt, temp=0.6, max_retries=1, timeout=80.0):
    for attempt in range(max_retries):
        try:
            completion = model_client.chat.completions.create(
                model="local-model",
                messages=[{"role": "user", "content": prompt}],
                temperature=temp,
                stream=False,
                timeout=timeout,
            )
            return completion.choices[0].message.content

        except (APITimeoutError, APIConnectionError, APIError) as e:
            print(f"[Attempt {attempt+1}/{max_retries}] Error: {e}. Retrying in 2 seconds...")
            time.sleep(2)

    print("[SKIPPED] Model did not respond in time. Skipping this sentence.")
    return None  # or "" if that's easier to handle later


In [35]:
def get_corrections_tokenized_with_labels_from_data_bert(sentence, model_client, max_attempts=2):
    tokens = [tok[1] for tok in sentence]
    sentence_str = " ".join(tokens)

    gold_correction = []
    error_types = []

    for tok in sentence:
        gold = tok[-2]
        pred = tok[-1]
        orig = tok[1]
        corr = tok[2]
        etype = tok[3] if tok[3] else "NO_LABEL"

        if gold == 'i':
            error_types.append(etype)
            if corr:
                gold_correction.append(corr)
            else: 
                continue
        else:
            gold_correction.append(orig)

    prompt = f"""You are a careful English grammatical corrector.

Given a tokenized sentence, return up to maximum five corrected versions. Each version should:
- Be minimally corrected (fix grammar and lexical errors only)
- Be tokenized (space-separated words)
- Be written on a separate line
- Use deletions if a word is clearly unnecessary or incorrect
- Do not add explanations
- If fewer than five unique corrections are possible, write an empty line for each missing correction

Input: {sentence_str}
Output:
"""

    for attempt in range(max_attempts):
        response = query_LLM(model_client, prompt)

        if response:
            lines = [line.strip() for line in response.strip().split("\n")]
            valid_corrections = [line.split() for line in lines if line.strip()]

            if valid_corrections:
                labels = []
                for corrected in valid_corrections:
                    label_seq = ["c" if i < len(tokens) and orig == corr else "i"
                                 for i, (orig, corr) in enumerate(zip(tokens, corrected))]
                    labels.append(label_seq)

                gold_labels = [tok[-2] for tok in sentence]
                pred_labels = [tok[-1] for tok in sentence]
                
                print("\nOriginal:    ", sentence_str)
                print("Gold labels: ", " ".join(gold_labels))
                print("Pred labels: ", " ".join(pred_labels))
                print("Gold:        ", " ".join(gold_correction))
                for idx, corr in enumerate(valid_corrections):
                    print(f"Correction {idx+1}:", " ".join(corr))
                print("Error types: ", list(set(error_types)))
                print()

                return {
                    "original": tokens,
                    "gold_correction": gold_correction,
                    "error_types": list(set(error_types)),
                    "llm_corrections": valid_corrections,
                    "labels": labels
                }
            else:
                print(f"Attempt {attempt+1}: No valid corrections, retrying...")
        else:
            print(f"Attempt {attempt+1}: Empty response, retrying...")

    print("\nOriginal:", sentence_str)
    print("No corrections found.\n")
    return {
        "original": tokens,
        "gold_correction": gold_correction,
        "error_types": list(set(error_types)),
        "llm_corrections": [],
        "labels": []
    }

# GENERATION BY ERROR TYPE - FCE- ROBERTA 

## FN

### ROBERTA --> TV

In [40]:
all_TV_roberta_corrections_FN = []

for sentence in tqdm(roberta_fce_only_fn_error_type['TV']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_TV_roberta_corrections_FN.append(sentence_corrections)

  1%|▍                                         | 1/91 [01:09<1:44:42, 69.80s/it]


Original:     The trip that you planned for us , students , it 's a great chance to meet new people and visit a beautiful city like London .
Gold labels:  c c c c i c c c c c i i c c c c c c c c c c c c c c c
Pred labels:  c c c c c c c c c c c c c c c c c c c c c c c c c c c
Gold:         The trip that you have planned for us , students , is is a great chance to meet new people and visit a beautiful city like London .
Correction 1: The trip that you planned for us, students, is a great chance to meet new people and visit a beautiful city like London.
Correction 2: The trip that you planned for us students it's a great chance to meet new people and visit a beautiful city like London
Correction 3: The trip that you planned for us students is a great chance to meet new people and visit a beautiful city like London
Correction 4: The trip that you planned for us, students, is a great chance to meet new people and visit a beautiful place like London
Correction 5: The trip that you planned 

  2%|▉                                         | 2/91 [02:03<1:29:17, 60.19s/it]


Original:     He had not liked this habit and had thought it was wrost of him until he solved a big crime .
Gold labels:  c i i i c c c c c c c i c c c c c c c c c
Pred labels:  c c c c c c c c c c c i c c c c c c c c c
Gold:         He did not like did not like did not like this habit and had thought it was worst of him until he solved a big crime .
Correction 1: He had not liked this habit and had thought it was worst of him until he solved a big crime.
Correction 2: He had not liked this habit and had thought it was the worst of him until he solved a big crime.
Correction 3: He had not liked this habit and had thought it was the worst until he solved a big crime.
Correction 4: He had not liked this habit and had thought it was worst until he solved a big crime.
Correction 5: He did not like this habit and thought it was the worst of him until he solved a big crime.
Error types:  ['S', 'TV']



  3%|█▍                                        | 3/91 [02:24<1:02:28, 42.59s/it]

...

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 1: Empty response, retrying...


[Attempt 1/1] Error: Connection error.. Retrying in 2 seconds...


100%|██████████| 91/91 [10:03<00:00,  6.66s/it]

100%|██████████| 91/91 [10:03<00:00,  6.63s/it]

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 2: Empty response, retrying...

Original: EVENTHOUGH YOUR PROGRAM IS PERFECT WE WILL PREFER TO GO THERE .
No corrections found.



In [46]:
all_AGV_roberta_corrections_FN = []

for sentence in tqdm(roberta_fce_only_fn_error_type['AGV']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_AGV_roberta_corrections_FN.append(sentence_corrections)

 10%|████▍                                       | 1/10 [00:40<06:00, 40.07s/it]


Original:     For the fashion of the future , I think it depends on the people 's own personalities and develope their own styles which suits them the most .
Gold labels:  i c c c c c c c c c c c c c c c c c i c c c c i c c c c
Pred labels:  i c c c c c c c c c c c c c c c c c i c c c c c c c c c
Gold:         Concerning the fashion of the future , I think it depends on the people 's own personalities and develop their own styles which suit them the most .
Correction 1: For the fashion of the future, I think it depends on people's own personalities and developing their own styles which suit them the most.
Correction 2: For the fashion of the future, I think it depends on people's own personalities and developing their own styles that suit them the most.
Correction 3: For the fashion of the future, I think it depends on people's own personalities and developing their own styles for them the most.
Correction 4: For the fashion of the future, I think it depends on people's own personalit

 20%|████████▊                                   | 2/10 [00:47<02:46, 20.84s/it]


Original:     Moreover there was no discount ticket .
Gold labels:  c c i c c i c
Pred labels:  c c c c c c c
Gold:         Moreover there were no discount tickets .
Correction 1: Moreover there was no discount ticket
Correction 2: Moreover there were no discount tickets
Correction 3: There was no discount ticket
Correction 4: No discount ticket
Correction 5: Moreover no discount ticket
Error types:  ['AGN', 'AGV']



 30%|█████████████▏                              | 3/10 [01:15<02:49, 24.28s/it]

...

100%|███████████████████████████████████████████| 10/10 [04:13<00:00, 25.39s/it]


Original:     I start thinking what she or he prefer and I try my best buying something apropiate does n't matter how much money does it cost .
Gold labels:  c c c i c c c i c c c c c i c i i i c c c c c c i c
Pred labels:  c c c c c c c i c c c c c c c i c c c c c c c c c c
Gold:         I start thinking she or he prefers and I try my best to buy something appropriate no no matter how much money does it costs .
Correction 1: I start thinking what she or he prefers and I try my best to buy something appropriate. Does it matter how much money it costs?
Correction 2: I start thinking about what she or he prefers and I try my best to buy something appropriate. Does it matter how much money it costs?
Correction 3: I start thinking what she or he prefers and I try my best buying something appropriate does n't matter how much money it costs.
Correction 4: I start thinking what she or he prefers and I try my best buying something appropriate does n't matter how much it costs.
Correction 5: I

100%|██████████| 10/10 [01:06<00:00,  6.61s/it]

100%|██████████| 10/10 [01:06<00:00,  6.63s/it]

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 2: Empty response, retrying...

Original: Moreover there was no discount ticket .
No corrections found.



In [47]:
all_AGN_roberta_corrections_FN = []

for sentence in tqdm(roberta_fce_only_fn_error_type['AGN']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_AGN_roberta_corrections_FN.append(sentence_corrections)

  5%|██▎                                         | 1/19 [00:23<06:57, 23.19s/it]


Original:     For all these reason , I want to ask for my money back and I hope to hear from you soon !
Gold labels:  c c c i c c c c c c c c c c c c c c c c c c
Pred labels:  c c c c c c c c c c c c c c c c c c c c c c
Gold:         For all these reasons , I want to ask for my money back and I hope to hear from you soon !
Correction 1: For all these reasons, I want to ask for my money back and I hope to hear from you soon!
Correction 2: For all these reasons, I want to ask for a refund and I hope to hear from you soon!
Correction 3: For all these reasons, I want to request a refund and I hope to hear from you soon!
Correction 4: For all these reasons, I want to ask for my money back and hopefully will hear from you soon!
Correction 5: For all these reasons, I want to ask for my money back and I hope to hear from you as soon as possible!
Error types:  ['AGN']



 11%|████▋                                       | 2/19 [01:07<10:01, 35.40s/it]


Original:     I would prefer an accomodation in log cabins because I am not keen on tents and I think that a log cabin is more comfortable than a tent .
Gold labels:  c c c i i c i i c c c c c c c c c c c c c c c c c c c c c
Pred labels:  c c c i i c c c c c c c c c c c c c c c c c c c c c c c c
Gold:         I would prefer to stay accommodation in log cabin log cabin because I am not keen on tents and I think that a log cabin is more comfortable than a tent .
Correction 1: I would prefer accommodation in log cabins because I am not keen on tents and I think that a log cabin is more comfortable than a tent.
Correction 2: I would prefer an accommodation in log cabins because I am not keen on tents and I think that a log cabin is more comfortable than a tent.
Correction 3: I would prefer an accommodation in log cabins because I am not keen on tents and I think log cabins are more comfortable than tents.
Correction 4: I would prefer an accommodation in log cabins because I am not keen on

 16%|██████▉                                     | 3/19 [01:21<06:51, 25.69s/it]

...

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 1: Empty response, retrying...


[Attempt 1/1] Error: Connection error.. Retrying in 2 seconds...


100%|██████████| 19/19 [02:06<00:00,  6.63s/it]

100%|██████████| 19/19 [02:06<00:00,  6.67s/it]

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 2: Empty response, retrying...

Original: As state in your letter I have to choose the accommodation type . I would prefer to stay in tents because since I was a child I have being using this type of accommodation .
No corrections found.



### ROBERTA --> RV

In [49]:
all_RV_roberta_corrections_FN = []

for sentence in tqdm(roberta_fce_only_fn_error_type['RV']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_RV_roberta_corrections_FN.append(sentence_corrections)

  1%|▍                                          | 1/105 [00:30<52:18, 30.18s/it]


Original:     The other activity I would like to take is tennis , I 'm quite good at it and I have been practising for four years .
Gold labels:  c c c c c c c i c c c c c c c c c c c c c i c c c c
Pred labels:  c c c c c c c c c c c c c c c c c c c c c c c c c c
Gold:         The other activity I would like to do is tennis , I 'm quite good at it and I have been playing for four years .
Correction 1: The other activity I would like to take is tennis. I'm quite good at it and have been practising for four years.
Correction 2: The other activities I would like to take are tennis, I'm quite good at it and have been practising for four years.
Correction 3: I would like to take another activity, which is tennis; I'm quite good at it and have been practising for four years.
Correction 4: Tennis is the other activity I would like to take, I'm quite good at it and have been practising for four years.
Correction 5: The other activity I would like to take is tennis; I am quite good at it and h

  2%|▊                                          | 2/105 [00:45<37:02, 21.57s/it]


Original:     But everyone always noticed he had secrets from his face .
Gold labels:  c c c i i i i i i i c
Pred labels:  c c c c c c c i c c c
Gold:         But everyone always saw from his face he had a secret from his face he had a secret secret .
Correction 1: But everyone always noticed that he had secrets from his face.
Correction 2: But everyone always noticed his secrets from his face.
Correction 3: But everyone always noticed the signs of his secrets in his face.
Correction 4: But everyone always noticed that he hid secrets in his face.
Correction 5: But everyone always noticed his hidden secrets in his face.
Error types:  ['AGN', 'NO_LABEL', 'RV', 'W']



  3%|█▏                                         | 3/105 [01:11<40:03, 23.56s/it]

...

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 1: Empty response, retrying...


[Attempt 1/1] Error: Connection error.. Retrying in 2 seconds...


100%|██████████| 105/105 [11:37<00:00,  6.60s/it]

100%|██████████| 105/105 [11:37<00:00,  6.65s/it]

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 2: Empty response, retrying...

Original: At that time he was unconcious , I kept asking myself ' should I wake him up or try to use my unsure first aid skill to deal with him .
No corrections found.



### ROBERTA --> SX

In [52]:
all_SX_roberta_corrections_FN = []

for sentence in tqdm(roberta_fce_only_fn_error_type['SX']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_SX_roberta_corrections_FN.append(sentence_corrections)

  9%|████                                        | 1/11 [00:11<01:57, 11.73s/it]


Original:     Tent will be more cooler to sleep then the cabin .
Gold labels:  i c c c c c c i i c c
Pred labels:  i c c c c c c c c c c
Gold:         A tent will be more cooler to sleep than a cabin .
Correction 1: Tent will be more cool to sleep than the cabin.
Correction 2: Tent will be cooler to sleep than the cabin.
Correction 3: Tents will be more cooler to sleep than the cabin.
Correction 4: Tent will be more cooled to sleep than the cabin.
Correction 5: Tent will be more cooling to sleep than the cabin.
Error types:  ['RD', 'SX', 'MD']



 18%|████████                                    | 2/11 [00:17<01:12,  8.09s/it]


Original:     I have qustions to .
Gold labels:  c c i i c
Pred labels:  c c i c c
Gold:         I have questions too .
Correction 1: I have questions to .
Correction 2: I have questions to .
Correction 3: I have questions to .
Correction 4: I have questions to .
Correction 5: I have questions to .
Error types:  ['SX', 'S']



 27%|████████████                                | 3/11 [00:45<02:17, 17.23s/it]

...


Original:     Propably in the future we might be to busy to go to our own home and spend some time there .
Gold labels:  i c c c c i c i c c c c c c c c c c c c c
Pred labels:  i c c c c c c c c c c c c c c c c c c c c
Gold:         Probably in the future we may be too busy to go to our own home and spend some time there .
Correction 1: Propably in the future we might be too busy to go to our own home and spend some time there.
Correction 2: Probably in the future we might be too busy to go to our own home and spend some time there.
Correction 3: Propably in the future we might be to busy to go to our home and spend some time there.
Correction 4: Probably in the future we might be to busy to go to our home and spend some time there.
Correction 5: Probably in the future we might be too busy to go to our own home and spend some time there.
Error types:  ['SX', 'S', 'TV']



[Attempt 1/1] Error: Connection error.. Retrying in 2 seconds...


100%|██████████| 11/11 [01:13<00:00,  6.66s/it]

100%|██████████| 11/11 [01:13<00:00,  6.66s/it]

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 2: Empty response, retrying...

Original: For example , I was born knowing the man has stepped on the moon .
No corrections found.



### ROBERTA --> w

In [54]:
all_W_roberta_corrections_FN = []

for sentence in tqdm(roberta_fce_only_fn_error_type['W']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_W_roberta_corrections_FN.append(sentence_corrections)

  2%|▋                                           | 1/66 [00:21<23:02, 21.26s/it]


Original:     First of all , I would like to tell you that I 've never had a such good time in my life !
Gold labels:  c c c c c c c c c c c c c c c i i c c c c c c
Pred labels:  c c c c c c c c c c c c c c c c c c c c c c c
Gold:         First of all , I would like to tell you that I 've never had such a such a good time in my life !
Correction 1: First of all, I would like to tell you that I've never had such a good time in my life!
Correction 2: First of all, I would like to tell you that I have never had such a good time in my life!
Correction 3: First of all, I would like to tell you that I've never had so good a time in my life!
Correction 4: First of all, I would like to tell you that I've never had such good a time in my life!
Correction 5: First of all, I would like to tell you that I've never had such a good time ever!
Error types:  ['W']



  3%|█▎                                          | 2/66 [00:43<23:17, 21.84s/it]


Original:     It is said that the singers are Danny Brook and Tina Truelove in the advertisement .
Gold labels:  c c c i i i i i i i i i i i i c
Pred labels:  c c c c c c c c c c c c c c c c
Gold:         It is said in the advertisement that the singers are Danny Brook and Tina Truelove in the advertisement that the singers are Danny Brook and Tina Truelove in the advertisement that the singers are Danny Brook and Tina Truelove in the advertisement that the singers are Danny Brook and Tina Truelove in the advertisement that the singers are Danny Brook and Tina Truelove in the advertisement that the singers are Danny Brook and Tina Truelove in the advertisement that the singers are Danny Brook and Tina Truelove in the advertisement that the singers are Danny Brook and Tina Truelove in the advertisement that the singers are Danny Brook and Tina Truelove in the advertisement that the singers are Danny Brook and Tina Truelove in the advertisement that the singers are Danny Brook and Tina 

  5%|██                                          | 3/66 [01:24<31:53, 30.37s/it]

...


Original:     I would like You to know also about my two hobbys : swimming and surfing .
Gold labels:  c c i i i i i c c c i c c c c c
Pred labels:  c c c i c c i c c c i c c c c c
Gold:         I would also like you to know you about my two hobbies : swimming and surfing .
Correction 1: I would like you to know also about my two hobbies: swimming and surfing.
Correction 2: I would like you to know about my two hobbys: swimming and surfing.
Correction 3: I would like you to know also about my two hobbys: swimming and surf.
Correction 4: I would like you to know also about my two hobbys : swimming and surf.
Correction 5: I would like you to know also about my two hobbies : swimming and surf.
Error types:  ['IN', 'NO_LABEL', 'RP', 'W']



 98%|██████████████████████████████████████████▎| 65/66 [23:11<00:15, 15.92s/it]


Original:     I had once this problem .
Gold labels:  c i i c c c
Pred labels:  c c i c c c
Gold:         I once had once had this problem .
Correction 1: I had this problem once .
Correction 2: I have once had this problem .
Correction 3: I had the problem once .
Correction 4: Once I had this problem .
Correction 5: I once had this problem .
Error types:  ['W']



100%|███████████████████████████████████████████| 66/66 [23:29<00:00, 21.35s/it]


Original:     When finally the show had started I found out that Danny Brook was replaced by another actor .
Gold labels:  c i i i c c c c c c c c i c c c c c
Pred labels:  c c c c c c c c c c c c c c c c c c
Gold:         When the show finally the show finally the show finally had started I found out that Danny Brook had been replaced replaced by another actor .
Correction 1: When finally the show had started, I found out that Danny Brook was replaced by another actor.
Correction 2: When the show had finally started, I found out that Danny Brook was replaced by another actor.
Correction 3: When the show had started, I found out that Danny Brook had been replaced by another actor.
Correction 4: When the show had started, I found out that Danny Brook was replaced by an actor.
Correction 5: When the show had started, I found out that Danny Brook was replaced by another performer.
Error types:  ['W', 'TV']



## FN+FP

### ROBERTA --> TV

In [57]:
all_TV_roberta_corrections_FN_FP = []

for sentence in tqdm(roberta_fce_fn_fp_error_type['TV']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_TV_roberta_corrections_FN_FP.append(sentence_corrections)

  3%|█▎                                          | 1/34 [00:22<12:27, 22.65s/it]


Original:     That might be a secret for Francis - and they decided the program and the present , a new gold watch .
Gold labels:  c i i c c c c c c c c i c c c c c c c c c c
Pred labels:  c c c c c c c c c c c c c c c i c c c c c c
Gold:         That should have been a secret for Francis - and they decided program and the present , a new gold watch .
Correction 1: That might be a secret for Francis - and they decided on the program and the present, a new gold watch.
Correction 2: That might be a secret for Francis - and they decided the program and the present, a new gold watch.
Correction 3: That might be a secret for Francis - and they decided the program and the present, a new gold watch.
Correction 4: That might be a secret for Francis - and they decided on the program and the present, a new gold watch.
Correction 5: That might be a secret for Francis - and they decided the program and the present, a new gold watch.
Error types:  ['NO_LABEL', 'RV', 'TV']



  6%|██▌                                         | 2/34 [00:37<09:46, 18.34s/it]


Original:     The fourth day , the singers and the players have been to there to test .
Gold labels:  c c c c c c c c i i i c c c c c
Pred labels:  i c c c c c c c c i c c c c c c
Gold:         The fourth day , the singers and the musicians were were to there to test .
Correction 1: The fourth day, the singers and players have been there to test.
Correction 2: The fourth day, the singers and the players have gone there to test.
Correction 3: The fourth day, the singers and the players had been there to test.
Correction 4: The fourth day, the singers and the players have been there for testing.
Correction 5: The fourth day, the singers and the players have been there to do tests.
Error types:  ['RN', 'TV']



  9%|███▉                                        | 3/34 [01:02<10:55, 21.15s/it]

...

100%|███████████████████████████████████████████| 34/34 [43:53<00:00, 77.45s/it]


Original:     I am really appreciate if you can answer me these doubt and saying again .
Gold labels:  c i c c i c i c c c i c c c c
Pred labels:  c c c i i c c c c i i c i c c
Gold:         I would really appreciate you could answer me these doubts and saying again .
Correction 1: I am really appreciate it if you can answer my doubts and say again.
Correction 2: I am really appreciative if you can answer my doubts and say again.
Correction 3: I am really appreciate if you can answer my doubts and say again.
Correction 4: I am really appreciate if you can answer these doubts and say again.
Correction 5: I am really appreciate if you can answer my doubts and saying it again.
Error types:  ['AGN', 'NO_LABEL', 'RV', 'TV']



100%|██████████| 34/34 [03:45<00:00,  6.65s/it]

100%|██████████| 34/34 [03:45<00:00,  6.64s/it]

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 2: Empty response, retrying...

Original: But , I 've got though it . I have to taken care of them during a break , served a drink , clothes , even brought them some cigaratte and anything they wants .
No corrections found.



### ROBERTA --> RV

In [59]:
all_RV_roberta_corrections_FN_FP = []

for sentence in tqdm(roberta_fce_fn_fp_error_type['RV']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_RV_roberta_corrections_FN_FP.append(sentence_corrections)

  3%|█▎                                          | 1/35 [00:25<14:31, 25.62s/it]


Original:     That might be a secret for Francis - and they decided the program and the present , a new gold watch .
Gold labels:  c i i c c c c c c c c i c c c c c c c c c c
Pred labels:  c c c c c c c c c c c c c c c i c c c c c c
Gold:         That should have been a secret for Francis - and they decided program and the present , a new gold watch .
Correction 1: That might be a secret for Francis - and they decided on the program and the present, a new gold watch.
Correction 2: That might be a secret for Francis - and they decided the program and the present, a new gold watch.
Correction 3: That might be a secret for Francis - and they decided the program and the present: a new gold watch.
Correction 4: That might be a secret for Francis - and they decided the program and the present, a gold watch.
Correction 5: That might be a secret for Francis - and they decided the program and the present - a new gold watch.
Error types:  ['NO_LABEL', 'RV', 'TV']

[Attempt 1/1] Error: Request t

  6%|██▎                                      | 2/35 [08:35<2:44:16, 298.68s/it]

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 2: Empty response, retrying...

Original: In other hand you ask me I have the chance for to choose two activities while I am at the camp . but is no easy , because all is very interesting for me .
No corrections found.

[Attempt 1/1] Error: Request timed out.. Retrying in 2 seconds...
[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 1: Empty response, retrying...
[Attempt 1/1] Error: Request timed out.. Retrying in 2 seconds...


  9%|███▌                                     | 3/35 [16:45<3:25:49, 385.92s/it]

...


Original:     I 've been recently seeing in your theatre during my week 's holiday in London I must say I had a very disappointing evening ! I was sure this was the London 's newest and best musical show I had ever seen because of the good publicity you made in your advertisement , but I thought completely wrong !
Gold labels:  c i i i i i c c c c c c c c c i c c c c c c c c c c c c c c c c c c c c c c i i i i c c c c c c i i c c c c c c c c c
Pred labels:  c i i i i i c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c i c c
Gold:         I I recently saw I recently saw I recently saw I recently saw your theatre during my week 's holiday in London must say I had a very disappointing evening ! I was sure this was the London 's newest and best musical show ever see ever see ever see because of the good publicity you gave your advertisement , but I thought completely wrong !
Correction 1: I've been recently seeing in your theatre during my

 97%|█████████████████████████████████████████▊ | 34/35 [49:32<00:28, 28.68s/it]


Original:     We thought , as usual we do in London theatre , we could have discounts as it 's written on your advertisement .
Gold labels:  c c c c i c c c c i c c c i c i c c c c c c c
Pred labels:  c c c c c c c c i i c c c c c c c c c i c c c
Gold:         We thought , as usually we do in London theatres , we could get discounts and it 's written on your advertisement .
Correction 1: We thought, as usual we do in London theatre, we could have discounts as it's written on your advertisement.
Correction 2: We thought, as usual we do at London theatre, we could have discounts as it's written on your advertisement.
Correction 3: We thought as usual we do in London theatre we could have discounts as it's written on your advertisement.
Correction 4: We thought as usual we do in London theatre that we could have discounts as it's written on your advertisement.
Correction 5: We thought as usual we do at London theatre that we could have discounts as it's written on your advertisement.
Err

100%|███████████████████████████████████████████| 35/35 [49:50<00:00, 85.44s/it]


Original:     I am really appreciate if you can answer me these doubt and saying again .
Gold labels:  c i c c i c i c c c i c c c c
Pred labels:  c c c i i c c c c i i c i c c
Gold:         I would really appreciate you could answer me these doubts and saying again .
Correction 1: I am really appreciate it if you can answer my doubts and say again.
Correction 2: I really appreciate if you can answer me these doubts and saying again.
Correction 3: I really appreciate it if you can answer my doubts and say again.
Correction 4: I am really appreciate it if you can answer my doubt and say again.
Correction 5: I am really appreciate if you can answer my doubt and say again.
Correction 6: (I am really appreciate it if you can answer my doubts and say again.)
Error types:  ['AGN', 'NO_LABEL', 'RV', 'TV']



### ROBERTA --> SX

In [61]:
all_SX_roberta_corrections_FN_FP = []

for sentence in tqdm(roberta_fce_fn_fp_error_type['SX']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_SX_roberta_corrections_FN_FP.append(sentence_corrections)

 12%|█████▋                                       | 1/8 [00:25<02:57, 25.42s/it]


Original:     First of all I want to congratulate your cousin with fifteenth birthday and with all the best .
Gold labels:  c c c c c c i i i i c c c i i c c c
Pred labels:  c c c c c c c c c i i c c c c c c c
Gold:         First of all I want to wish your cousin a happy fifteenth birthday wish your cousin a happy fifteenth birthday wish your cousin a happy fifteenth birthday on fifteenth birthday and wish the best .
Correction 1: First of all I want to congratulate your cousin on his fifteenth birthday and wish him all the best.
Correction 2: First of all I want to congratulate your cousin on her fifteenth birthday and wish her all the best.
Correction 3: First of all I want to congratulate your cousin on his or her fifteenth birthday and wish him or her all the best.
Correction 4: First of all I want to congratulate your cousin on his or her fifteenth birthday and with all the best.
Correction 5: First of all I want to congratulate your cousin on his or her fifteenth birthday.
Error

 25%|███████████▎                                 | 2/8 [00:52<02:40, 26.67s/it]


Original:     Pleas apologise me , but there were many mistakes and problems on the note you have made , which disappointed me very .
Gold labels:  i i c c c c c c c c c c c c c c c c c c c c i
Pred labels:  c c c c c c c c c c c c c c c i c c c c c c c
Gold:         Please excuse me , but there were many mistakes and problems on the note you have made , which disappointed me very
Correction 1: Please apologize me, but there were many mistakes and problems on the note you made, which disappointed me very much.
Correction 2: Please apologise me, but there were many mistakes and problems on the note you have made, which disappointed me very much.
Correction 3: Please apologise me, but there were many mistakes and problems in the note you made, which disappointed me very much.
Correction 4: Please apologize me, but there were many mistakes and problems in the note you have made, which disappointed me very much.
Correction 5: Please apologize me, but there were many mistakes and problems 

 38%|████████████████▉                            | 3/8 [01:15<02:03, 24.74s/it]

...


Original:     We do n't have to wear any special uniforms on signs of our school .
Gold labels:  c c c c c c c c c i c c c c c
Pred labels:  c c c c c c c c c c i c c c c
Gold:         We do n't have to wear any special uniforms or signs of our school .
Correction 1: We don't have to wear any special uniforms at our school.
Correction 2: We don't have to wear any special uniforms on our school.
Correction 3: We don't have to wear any special uniforms at signs of our school.
Correction 4: We don't have to wear any special uniforms for signs of our school.
Correction 5: We don't have to wear any special uniforms at our school signs.
Error types:  ['SX']



 88%|██████████████████████████████████████▌     | 7/8 [11:31<01:51, 111.95s/it]


Original:     The computer for example makes communication much easier and faster then in past time , when everything was send by common mail , and it took weeks till it got the right place .
Gold labels:  c c c c c c c c c c i c i i c c c c i c i c c c c i c c c c i c c c
Pred labels:  c c c c c c c c c c c c c c c c c c c c i c c c c c c c c i c c c c
Gold:         The computer for example makes communication much easier and faster than in times past times past , when everything was sent by normal mail , and it was weeks till it got right place .
Correction 1: The computer, for example, makes communication much easier and faster than in the past, when everything was sent by common mail, and it took weeks to get there.
Correction 2: The computer for example makes communication much easier and faster than in the past when everything was sent by common mail and it took weeks to get there.
Correction 3: The computer for example makes communication much easier and faster than in past tim

100%|█████████████████████████████████████████████| 8/8 [11:54<00:00, 89.28s/it]


Original:     They can be thrown away after wearing it and light enough to be carried around whenever you want to change your wearing .
Gold labels:  c c c c c c c i c i c c c c c c c c c c i c c
Pred labels:  c c c c c c c c c c c c c c c c c c c c c i c
Gold:         They can be thrown away after wearing them and enough to be carried around whenever you want to change you're wearing .
Correction 1: They can be thrown away after wearing it and light enough to be carried around whenever you want to change your outfit.
Correction 2: They can be thrown away after wearing them and light enough to be carried around whenever you want to change your wearing.
Correction 3: They can be thrown away after wearing them and light enough to be carried around whenever you want to change your clothes.
Correction 4: They can be discarded after wearing them and light enough to be carried around whenever you want to change your wearing.
Correction 5: They can be thrown away after wearing them and are l

In [62]:
all_AGV_roberta_corrections_FN_FP = []

for sentence in tqdm(roberta_fce_fn_fp_error_type['AGV']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_AGV_roberta_corrections_FN_FP.append(sentence_corrections)

 33%|███████████████                              | 1/3 [00:10<00:20, 10.27s/it]


Original:     Are there some possibility to rent the golfshoes ?
Gold labels:  i c i c c c c i c
Pred labels:  c c c i c c c c c
Gold:         Is there any possibility to rent the golf shoes ?
Correction 1: Are there some possibilities to rent the golf shoes?
Correction 2: Are there any possibility to rent the golf shoes?
Correction 3: Are there some possibility to rent the golf shoes?
Correction 4: Are there some possibilities to rent the golfshoes?
Correction 5: Is there any possibility to rent the golf shoes?
Error types:  ['RQ', 'AGV', 'RP']



 67%|██████████████████████████████               | 2/3 [00:43<00:23, 23.80s/it]


Original:     In other hand you ask me I have the chance for to choose two activities while I am at the camp . but is no easy , because all is very interesting for me .
Gold labels:  i c c c i c c c c c c c c c c c c c c c c c i i i c c c c i c c c c c
Pred labels:  i i c c c c c c c c c c c c c c c c c c c c i i i c c c i c c c c c c
Gold:         On the other hand other hand you tell me I have the chance for to choose two activities while I am at the camp . But not easy easy , because all are very interesting for me .
Correction 1: In other hand you ask me I have the chance for to choose two activities while I am at the camp. but is no easy, because all is very interesting for me.
Correction 2: In other hands you ask me I have the chance to choose two activities while I am at the camp. but is not easy, because all is very interesting for me.
Correction 3: In other hand you ask me I have the chance to choose two activities while I am at the camp. but it is no easy, because all is ver

100%|█████████████████████████████████████████████| 3/3 [01:40<00:00, 33.45s/it]


Original:     However , during my stay at Camp , there are two activities I can choose , one of them I will choose Basketball because it is the sport I play most , I and my friend used to win the first place in school - wide match when we was in school .
Gold labels:  i c c c c c c c c c c c c c c i c c c c c c c c c c c c c c c c c c c c c c c c c c c i c c c c c i i c c
Pred labels:  c c c c c c c c c c c c c c c i i c c c c c c c c c c c c c c c i c c c c c c c c c c c c i c c c c c c c
Gold:         Also , during my stay at Camp , there are two activities I can choose . For one of them I will choose Basketball because it is the sport I play most , I and my friend used to win the first place in - wide match when we were at school .
Correction 1: However, during my stay at Camp, there are two activities I can choose from. One of them I will choose Basketball because it is the sport I play most. I and my friend used to win the first place in school-wide matches when we were in school

In [63]:
all_AGN_roberta_corrections_FN_FP = []

for sentence in tqdm(roberta_fce_fn_fp_error_type['AGN']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_AGN_roberta_corrections_FN_FP.append(sentence_corrections)

 12%|█████▋                                       | 1/8 [00:40<04:40, 40.02s/it]


Original:     To spend the two weeks I would prefer to live in tents , that 's in my opinion a way to be nearer the enviroment and the animals , although I do n't mind living in log cabins .
Gold labels:  i i i i i i i i c c c i c c c c c c c c c c c c i c c c c c c c c c i c c c c
Pred labels:  c c c c c c c c c i c c i i c c c c c c c c c c i c c c c c c c c c c c c c c
Gold:         I would prefer to spend the two weeks I would prefer to spend the two weeks I would prefer to spend the two weeks I would prefer to spend the two weeks I would prefer to spend the two weeks I would prefer to spend the two weeks I would prefer to spend the two weeks I would prefer to spend the two weeks to live in tent , that 's in my opinion a way to be nearer the environment and the animals , although I do n't mind staying in log cabins .
Correction 1: To spend the two weeks, I would prefer to live in tents. That's in my opinion a way to be nearer to the environment and the animals. Although I don't mi

 25%|███████████▎                                 | 2/8 [01:02<02:56, 29.44s/it]


Original:     The reasons I am really writing , However , is to give you some details of my trip to California .
Gold labels:  c i c c c c c i c c c c c c c i c c c c c
Pred labels:  c c c c c c c i c i c c c c c c c c c c c
Gold:         The reason I am really writing , however , is to give you some details about my trip to California .
Correction 1: The reasons I am really writing, however, is to give you some details of my trip to California.
Correction 2: The reasons for which I am really writing, however, is to give you some details of my trip to California.
Correction 3: The reasons I am really writing, however, are to give you some details of my trip to California.
Correction 4: The reasons why I am really writing, however, is to give you some details of my trip to California.
Correction 5: The reasons why I am really writing, however, are to give you some details of my trip to California.
Error types:  ['AGN', 'RP', 'RT']



 38%|████████████████▉                            | 3/8 [01:44<02:56, 35.23s/it]

...


Original:     I would like to travel in July for two reason one 's is because I totaly will finish my exam in that date and the other in the first two week I will go to Farnborought is special 10 days the presentation the Air show and will be there the delegation from my country .
Gold labels:  c c c c c c c c c i i c c c c c c i c i i c i c c c i c c c i i c c c c c c c c c c c c c c i i i i i i i i c
Pred labels:  c c c c c c c c c c c c c c c i c c c c i c c c c c c c c c i c c c c i c c c c c c c c c c c c c i c c c c c
Gold:         I would like to travel in July for two reasons . One's 's is because I totaly will have finished my exams by that time and the other the first two weeks will go to Farnborought is special 10 days the presentation the Air show and the delegation from my country will be there the delegation from my country will be there the delegation from my country will be there the delegation from my country will be there the delegation from my country will be there 

 88%|███████████████████████████████████████▍     | 7/8 [08:04<01:07, 67.50s/it]


Original:     Before the concert actually started I had to give out lots of leaflets , they are list of pop band and group which will appear in the concert in order , and some information about their performance and backgrounds .
Gold labels:  c c c c c c c c c c c c c i i i i c c i c i c i c c c c c c c c c c c c c c c c
Pred labels:  c c c c c c c c i c c c c c c c i c i c c c c c c c c c c c c c c c c c c c c c
Gold:         Before the concert actually started I had to give out lots of leaflets . They . They were lists of pop bands and groups which would appear in the concert in order , and some information about their performance and backgrounds .
Correction 1: Before the concert actually started, I had to give out lots of leaflets. They are lists of pop bands and groups which will appear at the concert in order, and some information about their performances and backgrounds.
Correction 2: Before the concert actually started, I had to give out lots of leaflets. They are lists of po

100%|█████████████████████████████████████████████| 8/8 [08:26<00:00, 63.30s/it]


Original:     There were n't any discount available as you 'd assured in the add .
Gold labels:  c c c c i c c c c c i c i c
Pred labels:  c c c c c c c c c i c c i c
Gold:         There were n't any discounts available as you 'd assured the ad .
Correction 1: There weren't any discounts available as you'd assured in the ad.
Correction 2: There weren't any discounts available as you'd assured in the add.
Correction 3: There were no discounts available as you'd assured in the ad.
Correction 4: There were no discounts available as you'd assured in the add.
Correction 5: There were no discounts available as you'd assured in the advertisement.
Correction 6: Note: The fifth line is left empty as only four unique corrections were possible given the input sentence structure and constraints.
Error types:  ['AGN', 'SX', 'NO_LABEL']



In [64]:
all_W_roberta_corrections_FN_FP = []

for sentence in tqdm(roberta_fce_fn_fp_error_type['W']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_W_roberta_corrections_FN_FP.append(sentence_corrections)

  8%|███▍                                        | 1/13 [00:37<07:35, 37.96s/it]


Original:     Firstly , I 'm writing to thank you the great opportunity you give us , especially in planing all this programme in a such acurate way .
Gold labels:  c c c c c c c c i c c c i c c c c i c c c c i i i c c
Pred labels:  c c c c c c c c i c c c i c c c i i c c c c c c i c c
Gold:         Firstly , I 'm writing to thank you great opportunity you are giving us , especially in planning all this programme in such  such  accurate way .
Correction 1: Firstly, I'm writing to thank you for the great opportunity you give us, especially in planning all this programme in such an accurate way.
Correction 2: Firstly, I'm writing to thank you for the great opportunity you have given us, especially in planning all this programme in a such accurate way.
Correction 3: Firstly, I'm writing to thank you for the great opportunity you have given us, especially in planning all this program in a such accurate way.
Correction 4: Firstly, I'm writing to thank you for the great opportunity you gave

 15%|██████▊                                     | 2/13 [01:01<05:21, 29.20s/it]


Original:     I m not so good as her , but I 'm going every week once to her training .
Gold labels:  c i c c c c c c c c i i i i i c c c i
Pred labels:  c i c c c c c c c c c c c c c i c c c
Gold:         I I'm not so good as her , but I I go I go once every week once every week once every week to her training
Correction 1: I am not as good as her, but I go to her training once a week.
Correction 2: I am not so good as her, but I go to her training every week.
Correction 3: I am not so good as her, but I go to her training once a week.
Correction 4: I am not as good as her, but I go to her training.
Correction 5: I am not so good as her, but I go every week to her training.
Error types:  ['NO_LABEL', 'RP', 'TV', 'W']



 23%|██████████▏                                 | 3/13 [01:33<05:07, 30.79s/it]

...

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 2: Empty response, retrying...

Original: I arrived at the theatre in time and the musical show started later than I expected so I was getting nervous because I dislike very much the impuntuality , but that was not all .
No corrections found.



 92%|██████████████████████████████████████▊   | 12/13 [22:06<03:40, 220.54s/it]


Original:     I will only be able to set off in July and I also will give my thesis to my teacher in August .
Gold labels:  c c c c c c c c c c c i i c c c c c c c c c c
Pred labels:  c c c c c c c c c c c c i i c c c c c c c c c
Gold:         I will only be able to set off in July and also I also I will give my thesis to my teacher in August .
Correction 1: I will only be able to set off in July and I will also give my thesis to my teacher in August.
Correction 2: I will only be able to set out in July and I will also give my thesis to my teacher in August.
Correction 3: I will only be able to leave in July and I will also give my thesis to my teacher in August.
Correction 4: I will only be able to depart in July and I will also give my thesis to my teacher in August.
Correction 5: I will only be able to set off in July and I will give my thesis to my teacher in August.
Error types:  ['W']



100%|██████████████████████████████████████████| 13/13 [23:12<00:00, 107.10s/it]


Original:     I would like to travel in July for two reason one 's is because I totaly will finish my exam in that date and the other in the first two week I will go to Farnborought is special 10 days the presentation the Air show and will be there the delegation from my country .
Gold labels:  c c c c c c c c c i i c c c c c c i c i i c i c c c i c c c i i c c c c c c c c c c c c c c i i i i i i i i c
Pred labels:  c c c c c c c c c c c c c c c i c c c c i c c c c c c c c c i c c c c i c c c c c c c c c c c c c i c c c c c
Gold:         I would like to travel in July for two reasons . One's 's is because I totaly will have finished my exams by that time and the other the first two weeks will go to Farnborought is special 10 days the presentation the Air show and the delegation from my country will be there the delegation from my country will be there the delegation from my country will be there the delegation from my country will be there the delegation from my country will be there 

# GENERATION FOR FP ONLY - FCE - ROBERTA

In [66]:
#### all_false_positive_roberta = []  79 labels corretti mentre 49 errori di labels

for sentence in tqdm(roberta_fce_only_fp):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_false_positive_roberta.append(sentence_corrections)

  1%|▎                                        | 1/129 [00:29<1:03:33, 29.79s/it]


Original:     Dear Principal , First of all , we would like to thank you for good programme which provides us 3 days stay in London .
Gold labels:  c c c c c c c c c c c c c c i c c c c i c c c c c
Pred labels:  c c c c c c c c c c c c c c i c c c c i i i c c c
Gold:         Dear Principal , First of all , we would like to thank you for programme which provides us days stay in London .
Correction 1: Dear Principal, First of all, we would like to thank you for the good programme which provides us with a 3-day stay in London.
Correction 2: Dear Principal, First of all, we would like to thank you for the good programme that provides us with a 3-day stay in London.
Correction 3: Dear Principal, First of all, we would like to thank you for the good programme providing us with a 3-day stay in London.
Correction 4: Dear principal, first of all, we would like to thank you for the good programme which provides us with a 3-day stay in london.
Correction 5: Dear principal, first of all, we would

  2%|▋                                        | 2/129 [01:09<1:16:00, 35.91s/it]


Original:     However , we have found an advertisement for the London Fashion and Leisure show , which will be held at Central Exhibition Hall on 14th of March .
Gold labels:  c c c c c c c c c c c c c c c c c c c c c c c c c c c c
Pred labels:  c c c c c c c c c c c c c c c c c c c c i c c c i c c c
Gold:         However , we have found an advertisement for the London Fashion and Leisure show , which will be held at Central Exhibition Hall on 14th of March .
Correction 1: However, we have found an advertisement for the London Fashion and Leisure show, which will be held at Central Exhibition Hall on March 14th.
Correction 2: However, we have found an advertisement for the London Fashion and Leisure Show, which will be held at the Central Exhibition Hall on 14th of March.
Correction 3: However, we have found an advertisement for the London Fashion and Leisure Show, which will be held at the Central Exhibition Hall on March 14th.
Correction 4: However, we have found an ad for the Londo

  2%|▉                                        | 3/129 [01:41<1:10:58, 33.80s/it]

...

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 1: Empty response, retrying...


[Attempt 1/1] Error: Connection error.. Retrying in 2 seconds...


100%|██████████| 129/129 [14:16<00:00,  6.64s/it]

100%|██████████| 129/129 [14:16<00:00,  6.64s/it]

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 2: Empty response, retrying...

Original: Please , if you would be so kind and mind correct this mistake . I would be gratefull if you could correct too that show start at 20.15 not on 19.31 .
No corrections found.



# GENERATION FOR FN ONLY - FCE - ROBERTA

In [ ]:
random.seed(41)

sampled_sentences_roberta = random.sample(roberta_fce_only_fn, 130)

all_false_negative_roberta = []


for sentence in tqdm(sampled_sentences_roberta):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data_bert(sentence, client)
    all_false_negative_roberta.append(sentence_corrections)

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 2: Empty response, retrying...

Original: When all money had gone from a bank , nobady there except Pat .
No corrections found.



[Attempt 1/1] Error: Connection error.. Retrying in 2 seconds...


[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 1: Empty response, retrying...


[Attempt 1/1] Error: Connection error.. Retrying in 2 seconds...


100%|██████████| 130/130 [14:23<00:00,  6.62s/it]

100%|██████████| 130/130 [14:23<00:00,  6.64s/it]

[SKIPPED] Model did not respond in time. Skipping this sentence.
Attempt 2: Empty response, retrying...

Original: After that , I had to help people who had a ticket but ca n't find their seat .
No corrections found.

